In [ ]:
import os, sys
import torch
import datetime
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
import IPython.display as ipd
from argparse import Namespace
import torch.nn as nn
import torch.nn.functional as F
import librosa
import argparse
import scipy
import IPython.display as ipd
import numpy as np
from matplotlib.pyplot import cm
from tqdm import tqdm
import time 
import comet_ml
from pprint import pprint
import pandas as pd
from glob2 import glob
import torch
import os
import sys
import copy 
import IPython
import numpy as np
from asteroid.losses import pairwise_neg_sisdr
from asteroid.losses import pairwise_neg_snr
import BaseLine.DatasetLoaders.Chime as Chime
import BaseLine.Utils.MixtureConsistency as MixtureConsistency
import BaseLine.Models.ImprovedSudormrf as ImprovedSudormrf
import BaseLine.Metrics.DNNMosMetric as DNNMosMetric
import pickle
sys.path.append("../")
os.environ["CUDA_VISIBLE_DEVICES"]= ",".join(["2"])

In [ ]:
batch_size = 1
sample_rate = 16000
timelength = 4.
fixed_n_sources = 1
get_only_active_speakers = False
split = 'dev'
random_order = True
n_samples = 250
time_samples = int (sample_rate * timelength)
data_loader = Chime.Dataset(
    sample_rate = sample_rate, 
    fixed_n_sources = fixed_n_sources, 
    timelength = timelength, 
    augment = random_order,
    zero_pad = True, 
    split = split, 
    get_only_active_speakers = get_only_active_speakers, 
    normalize_audio = False, 
    n_samples = n_samples
)
eval_chime_gen = data_loader.get_generator(batch_size=batch_size, num_workers=1) 
def get_new_student(hparams, depth_growth):
    student = ImprovedSudormrf.SuDORMRF(
        out_channels = hparams["out_channels"], 
        in_channels = hparams["in_channels"], 
        num_blocks = int(depth_growth * hparams["num_blocks"]), 
        upsampling_depth = hparams["upsampling_depth"],
        enc_kernel_size = hparams["enc_kernel_size"], 
        enc_num_basis = hparams["enc_num_basis"], 
        num_sources = 2,
    )
    return student
hparams = {
    'out_channels': 512,
    'in_channels': 512,
    'num_blocks': 8,
    'upsampling_depth': 7,
    'enc_kernel_size': 41,
    'enc_num_basis': 512,
}
wo_mix_con_chkpt = "/home/thymios/projects/unsup_speech_enh_adaptation/pretrained_checkpoints/libri1to3_sup_teacher.pt"
wo_mix_con_model = get_new_student(hparams, depth_growth=1)
wo_mix_con_model.load_state_dict(torch.load(wo_mix_con_chkpt))
wo_mix_con_model = wo_mix_con_model.cuda()

In [ ]:
wo_mix_con_model.eval()
for cnt, mixture in enumerate(eval_chime_gen):
    input_mix = mixture.unsqueeze(1).cuda()
    input_mix_std = input_mix.std(-1, keepdim = True)
    input_mix_mean = input_mix.mean(-1, keepdim = True)
    input_mix = (input_mix - input_mix_mean) / (input_mix_std + 1e-9)
    with torch.no_grad():
        rec_sources_wavs = wo_mix_con_model(input_mix)
        new_mix = rec_sources_wavs[:, 0:1] + rec_sources_wavs[:, 1:]
        new_mix_std = new_mix.std(-1, keepdim = True)
        new_mix_mean = new_mix.mean(-1, keepdim = True)
        rec_sources_wavs = (rec_sources_wavs - new_mix_mean) / (new_mix_std + 1e-9)
        teacher_est_active_speakers = rec_sources_wavs[:, 0:1].detach().cpu()
        teacher_est_noises = rec_sources_wavs[:, 1:].detach().cpu()
    #Play the mixture and the estimated sources
    mix, est_s, est_n = input_mix[0, 0].cpu().numpy(), teacher_est_active_speakers[0, 0].cpu().numpy(), teacher_est_noises[0, 0].cpu().numpy()
    print("Initial Mixture")
    ipd.display(ipd.Audio(input_mix[0:, 0].cpu().numpy(), rate = 16000))
    print("Estimated Teacher Speech")
    ipd.display(ipd.Audio(teacher_est_active_speakers[0:, 0].detach().numpy(), rate = 16000))
    print("Teacher Estimated Noise")
    ipd.display(ipd.Audio(teacher_est_noises[0:, 0].detach().numpy(), rate = 16000))
    #Plot the mixture and the estimated sources
    fig, ax = plt.subplots(1, 3, figsize = (15,5), sharey = True)
    ax[0].plot(mix, label = "mixture")
    ax[1].plot(est_s, label = "est speaker mix")
    ax[2].plot(est_n, label = "est noise")
    plt.legend()
    plt.show()
    plt.close()
    #MEasure the DNSMOS
    dnsmos_val = DNNMosMetric.compute_dnsmos(est_s, fs = 16000)
    print(dnsmos_val)
    if cnt > 10:
        break

In [ ]:
#Extract proper numbers for val and test for DNS-MOS
eval_res_dic = {'sig_mos': [], 'bak_mos': [], 'ovr_mos': []}
wo_mix_con_model.eval()
for cnt, mixture in tqdm(enumerate(eval_chime_gen)):
    input_mix = mixture.unsqueeze(1).cuda()
    input_mix_std = input_mix.std(-1, keepdim = True)
    input_mix_mean = input_mix.mean(-1, keepdim = True)
    input_mix = (input_mix - input_mix_mean) / (input_mix_std + 1e-9)
    with torch.no_grad():
        rec_sources_wavs = wo_mix_con_model(input_mix)
        new_mix = rec_sources_wavs[:, 0:1] + rec_sources_wavs[:, 1:]
        new_mix_std = new_mix.std(-1, keepdim = True)
        new_mix_mean = new_mix.mean(-1, keepdim = True)
        rec_sources_wavs = (rec_sources_wavs - new_mix_mean) / (new_mix_std + 1e-9)
        teacher_est_active_speakers = rec_sources_wavs[:, 0:1].detach().cpu()
        teacher_est_noises = rec_sources_wavs[:, 1:].detach().cpu()
    mix, est_s, est_n = input_mix[0, 0].cpu().numpy(), teacher_est_active_speakers[0, 0].cpu()
    #MEasure the DNSMOS
    dnsmos_val = DNNMosMetric.compute_dnsmos(est_s, fs=16000)
    for k, v in dnsmos_val.items():
        eval_res_dic[k].append(v)
    if cnt > 250:
        break
val_res_dic = {'sig_mos': [], 'bak_mos': [], 'ovr_mos': []}
wo_mix_con_model.eval()
for cnt, mixture in tqdm(enumerate(val_chime_gen)):
    input_mix = mixture.unsqueeze(1).cuda()
    input_mix_std = input_mix.std(-1, keepdim = True)
    input_mix_mean = input_mix.mean(-1, keepdim = True)
    input_mix = (input_mix - input_mix_mean) / (input_mix_std + 1e-9)
    with torch.no_grad():
        rec_sources_wavs = wo_mix_con_model(input_mix)
        new_mix = rec_sources_wavs[:, 0:1] + rec_sources_wavs[:, 1:]
        new_mix_std = new_mix.std(-1, keepdim = True)
        new_mix_mean = new_mix.mean(-1, keepdim = True)
        rec_sources_wavs = (rec_sources_wavs - new_mix_mean) / (new_mix_std + 1e-9)
        teacher_est_active_speakers = rec_sources_wavs[:, 0:1].detach().cpu()
        teacher_est_noises = rec_sources_wavs[:, 1:].detach().cpu()
        mix, est_s, est_n = input_mix[0, 0].cpu().numpy(), teacher_est_active_speakers[0, 0].cpu().numpy(), teacher_est_noises[0, 0].cpu().numpy()
    #MEasure the DNSMOS
    dnsmos_val = DNNMosMetric.compute_dnsmos(est_s, fs=16000)
    for k, v in dnsmos_val.items():
        val_res_dic[k].append(v)
    if cnt > 250:
        break

In [ ]:
for name, this_dic in [('val', val_res_dic), ('eval', eval_res_dic)]:
    print(name)
    for k, v in this_dic.items():
        print(k, np.median(v))

In [ ]:
batch_size = 1
sample_rate = 16000
timelength = 4.
fixed_n_sources = 1
get_only_active_speakers = False
split = 'train'
random_order = False
n_samples = -1
time_samples = int(sample_rate * timelength)
data_loader = Chime.Dataset( 
    sample_rate = sample_rate, 
    fixed_n_sources = fixed_n_sources, 
    timelength = timelength, 
    augment = random_order,
    zero_pad = True, 
    split = split, 
    get_only_active_speakers = get_only_active_speakers, 
    normalize_audio = False, 
    n_samples = n_samples
)
train_chime_gen = data_loader.get_generator(batch_size = batch_size, num_workers = 1) 
for i in range(2):
    initial_seed = 11
    torch.manual_seed(initial_seed + i)
    np.random.seed(initial_seed + i)
    wo_mix_con_model.eval()
    for cnt, mixture in enumerate(train_chime_gen):
        input_mix = mixture.unsqueeze(1).cuda()
        input_mix_std = input_mix.std(-1, keepdim = True)
        input_mix_mean = input_mix.mean(-1, keepdim = True)
        input_mix = (input_mix - input_mix_mean) / (input_mix_std + 1e-9)
        with torch.no_grad():
            rec_sources_wavs = wo_mix_con_model(input_mix)
            new_mix = rec_sources_wavs[:, 0:1] + rec_sources_wavs[:, 1:]
            new_mix_std = new_mix.std(-1, keepdim = True)
            new_mix_mean = new_mix.mean(-1, keepdim = True)
            rec_sources_wavs = (rec_sources_wavs - new_mix_mean) / (new_mix_std + 1e-9)
            teacher_est_active_speakers = rec_sources_wavs[:, 0:1].detach().cpu()
            teacher_est_noises = rec_sources_wavs[:, 1:].detach().cpu()
        #Play the mixture and the estimated sources
        mix, est_s, est_n = input_mix[0, 0].cpu().numpy(), teacher_est_active_speakers[0, 0].cpu().numpy(), teacher_est_noises[0, 0].cpu().numpy()
        print("Initial Mixture")
        ipd.display(ipd.Audio(input_mix[0:, 0].cpu().numpy(), rate = 16000))
        print("Estimated Teacher Speech")
        ipd.display(ipd.Audio(teacher_est_active_speakers[0:, 0].detach().numpy(), rate=16000))
        print("Teacher Estimated Noise")
        ipd.display(ipd.Audio(teacher_est_noises[0:, 0].detach().numpy(), rate = 16000))
        #Plot the mixture and the estimated sources
        fig, ax = plt.subplots(1, 3, figsize = (15,5), sharey = True)
        ax[0].plot(mix, label="mixture")
        ax[1].plot(est_s, label="est speaker mix")
        ax[2].plot(est_n, label="est noise")
        plt.legend()
        plt.show()
        plt.close()
        #MEasure the DNSMOS
        dnsmos_val = DNNMosMetric.compute_dnsmos(est_s, fs = 16000)
        print(dnsmos_val)
        if cnt > -1:
            break

In [ ]:
#Test a model with mixture consistency
batch_size = 1
sample_rate = 16000
timelength = 4.
fixed_n_sources = 1
get_only_active_speakers = False
split = 'dev'
random_order = False
n_samples = 250
time_samples = int(sample_rate * timelength)
data_loader = Chime.Dataset(
    sample_rate = sample_rate, 
    fixed_n_sources = fixed_n_sources, 
    timelength = timelength, 
    augment = random_order,
    zero_pad = True, 
    split = split, 
    get_only_active_speakers = get_only_active_speakers, 
    normalize_audio = False, 
    n_samples = n_samples
)
val_chime_gen = data_loader.get_generator(batch_size = batch_size, num_workers=1) 
data_loader = Chime.Dataset(
    sample_rate = sample_rate, 
    fixed_n_sources = fixed_n_sources, 
    timelength = timelength, augment = random_order,
    zero_pad=True, split='eval', 
    get_only_active_speakers=get_only_active_speakers, 
    normalize_audio=False, 
    n_samples = n_samples
)
eval_chime_gen = data_loader.get_generator(batch_size=batch_size, num_workers=1) 
def get_new_student(hparams, depth_growth):
    student = ImprovedSudormrf.SuDORMRF(
        out_channels = hparams["out_channels"], 
        in_channels = hparams["in_channels"], 
        num_blocks = int(depth_growth * hparams["num_blocks"]), 
        upsampling_depth = hparams["upsampling_depth"], 
        enc_kernel_size = hparams["enc_kernel_size"], 
        enc_num_basis = hparams["enc_num_basis"], 
        num_sources =2,
    )
    return student
hparams = {
    'out_channels': 512,
    'in_channels': 512,
    'num_blocks': 8,
    'upsampling_depth': 7,
    'enc_kernel_size': 41,
    'enc_num_basis': 512,
}
w_mix_con_chkpt = "/mnt/data/thymios_backup/thymios/UCHIME_checkpoints/ublock8_sup_teacher_libri1to3mix_0.5single_w_mixcon/sup_teacher_epoch_100.pt"
w_mix_con_model = get_new_student(hparams, depth_growth=1)
w_mix_con_model.load_state_dict(torch.load(w_mix_con_chkpt))
w_mix_con_model = w_mix_con_model.cuda()

In [ ]:
# Extract proper numbers for val and test for DNS-MOS
eval_res_dic = {'sig_mos': [], 'bak_mos': [], 'ovr_mos': []}
w_mix_con_model.eval()
for cnt, mixture in tqdm(enumerate(eval_chime_gen)):
    input_mix = mixture.unsqueeze(1).cuda()
    input_mix_std = input_mix.std(-1, keepdim=True)
    input_mix_mean = input_mix.mean(-1, keepdim=True)
    input_mix = (input_mix - input_mix_mean) / (input_mix_std + 1e-9)
    with torch.no_grad():
        rec_sources_wavs = w_mix_con_model(input_mix)
        rec_sources_wavs = MixtureConsistency.apply(rec_sources_wavs, input_mix)
        new_mix = rec_sources_wavs[:, 0:1] + rec_sources_wavs[:, 1:]
        new_mix_std = new_mix.std(-1, keepdim=True)
        new_mix_mean = new_mix.mean(-1, keepdim=True)
        rec_sources_wavs = (rec_sources_wavs - new_mix_mean) / (new_mix_std + 1e-9)
        teacher_est_active_speakers = rec_sources_wavs[:, 0:1].detach().cpu()
        teacher_est_noises = rec_sources_wavs[:, 1:].detach().cpu()
    mix, est_s, est_n = input_mix[0, 0].cpu().numpy(), teacher_est_active_speakers[0, 0].cpu().numpy(), teacher_est_noises[0, 0].cpu().numpy()
    #MEasure the DNSMOS
    dnsmos_val = DNNMosMetric.compute_dnsmos(mix, fs=16000)
    for k, v in dnsmos_val.items():
        eval_res_dic[k].append(v)
    if cnt > 250:
        break
val_res_dic = {'sig_mos': [], 'bak_mos': [], 'ovr_mos': []}
wo_mix_con_model.eval()
for cnt, mixture in tqdm(enumerate(val_chime_gen)):
    input_mix = mixture.unsqueeze(1).cuda()
    input_mix_std = input_mix.std(-1, keepdim = True)
    input_mix_mean = input_mix.mean(-1, keepdim = True)
    input_mix = (input_mix - input_mix_mean) / (input_mix_std + 1e-9)
    with torch.no_grad():
        rec_sources_wavs = w_mix_con_model(input_mix)
        rec_sources_wavs = MixtureConsistency.apply(rec_sources_wavs, input_mix)
        new_mix = rec_sources_wavs[:, 0:1] + rec_sources_wavs[:, 1:]
        new_mix_std = new_mix.std(-1, keepdim = True)
        new_mix_mean = new_mix.mean(-1, keepdim = True)
        rec_sources_wavs = (rec_sources_wavs - new_mix_mean) / (new_mix_std + 1e-9)
        teacher_est_active_speakers = rec_sources_wavs[:, 0:1].detach().cpu()
        teacher_est_noises = rec_sources_wavs[:, 1:].detach().cpu()
    mix, est_s, est_n = input_mix[0, 0].cpu().numpy(), teacher_est_active_speakers[0, 0].cpu().numpy(), teacher_est_noises[0, 0].cpu().numpy()
    # MEasure the DNSMOS
    dnsmos_val = DNNMosMetric.compute_dnsmos(mix, fs=16000)
    for k, v in dnsmos_val.items():
        val_res_dic[k].append(v)
    
    if cnt > 250:
        break

In [ ]:
for name, this_dic in [('val', val_res_dic), ('eval', eval_res_dic)]:
    print(name)
    for k, v in this_dic.items():
        print(k, np.median(v))